# Prototipo RAG para TurnosMedicos

Este notebook implementa un prototipo básico de RAG (Retrieval Augmented Generation) para el sistema TurnosMedicos.

In [ ]:
import os
from typing import List
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

# Cargar variables de entorno (API Keys)
load_dotenv()

## 1. Cargar Documentos
Cargamos los archivos markdown de la carpeta `../docs`.

In [ ]:
DOCS_PATH = "../docs"

loader = DirectoryLoader(DOCS_PATH, glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()
print(f"Cargados {len(documents)} documentos.")

## 2. Split de Texto
Dividimos los documentos en chunks manejables.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
texts = text_splitter.split_documents(documents)
print(f"Generados {len(texts)} chunks.")

## 3. Embeddings y Vector Store
Usamos `sentence-transformers` para crear embeddings locales y FAISS para el índice.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(texts, embeddings)
print("Índice vectorial creado.")

## 4. Prueba de Búsqueda (Retrieval)
Probamos recuperar documentos relevantes para una pregunta.

In [ ]:
query = "¿Qué significa que un turno esté pendiente?"
docs = db.similarity_search(query, k=2)
for doc in docs:
    print("---")
    print(doc.page_content)

## 5. Generación con LLM (Opcional)
Si tienes una API Key de OpenAI, puedes probar la generación completa.

In [ ]:
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
# response = qa.run(query)
# print(response)